In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery

client = bigquery.Client()
dataset_ref = client.dataset("epa_historical_air_quality", project = "bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

In [ ]:
query = """
    WITH f AS 
           (
           SELECT
                county_name,
                state_name,
                date_local
            FROM
                `bigquery-public-data.epa_historical_air_quality.o3_daily_summary`
            WHERE
                 (aqi >= 101) #using days where the air was unhealthy for the whole population did not return enough results
                  AND (EXTRACT(YEAR from date_local) = 2012 OR EXTRACT(YEAR from date_local) = 2013 OR EXTRACT(YEAR from date_local) = 2014
                  OR EXTRACT(YEAR from date_local) = 2015 OR EXTRACT(YEAR from date_local) = 2016) 
                 
            )   
            
    SELECT
        CONCAT(b.county_name, " ", b.state_name) AS specific_county,
        COUNT(DISTINCT f.date_local) AS bad_days,
        AVG(c.arithmetic_mean) AS average_temperature,
        MAX(b.arithmetic_mean) AS worst_day_pollution,
        AVG(b.arithmetic_mean) AS average_o3,
        AVG(b.aqi) AS average_aqi,
        AVG(b.first_max_value) AS average_daily_peak_pollution,
        MAX(b.first_max_value) AS maximum_pollution_level,
        AVG(b.first_max_hour) AS average_daily_peak_pollution_time

    FROM
        `bigquery-public-data.epa_historical_air_quality.o3_daily_summary` AS b
        FULL OUTER JOIN
            `bigquery-public-data.epa_historical_air_quality.temperature_daily_summary` AS c
                ON
                    b.county_name = c.county_name AND b.date_local = c.date_local AND b.state_name = c.state_name
            FULL OUTER JOIN
                f AS f
                    ON 
                        c.county_name = f.county_name AND c.date_local = f.date_local AND c.state_name = f.state_name


   
    WHERE
        (EXTRACT(YEAR from b.date_local) = 2012 OR EXTRACT(YEAR from b.date_local) = 2013
        OR EXTRACT(YEAR from b.date_local) = 2014 OR EXTRACT(YEAR from b.date_local) = 2015 
        OR EXTRACT(YEAR from b.date_local) = 2016)
        
    GROUP BY
        specific_county
    ORDER BY
        average_o3 DESC
        

"""

In [ ]:
THREE_GB = 1000*1000*1000*3
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=THREE_GB)
safe_query_job = client.query(query, job_config=safe_config)

air_pollution_data = safe_query_job.to_dataframe()
air_pollution_data = air_pollution_data[air_pollution_data.specific_county.notnull()]
air_pollution_data['specific_county'] = air_pollution_data.specific_county.astype(str)
print("The shape of the air pollution dataset is: ") 
air_pollution_data.shape

In [ ]:
cancer_data = pd.read_csv("../input/cancer-incidence-totals-and-rates-per-us-county/cancer_incidence_by_county.csv")
cancer_data.county = cancer_data.county.astype(str)
cancer_data.county = cancer_data.county.str.replace(" County", "")
cancer_data.county = cancer_data.county.str.replace("7,8", "")
cancer_data.county = cancer_data.county.apply(lambda x: x.strip())
print("The shape of the cancer dataset is: ") 
cancer_data.shape

In [ ]:
cancer_data.loc[cancer_data.five_year_incidence_change_rate == "*"]
cancer_data['five_year_incidence_change_rate'] = cancer_data.five_year_incidence_change_rate.replace('*', np.nan)
cancer_data['five_year_incidence_change_rate'] = cancer_data.five_year_incidence_change_rate.replace('¶', np.nan)
cancer_data['five_year_incidence_change_rate'] = cancer_data.five_year_incidence_change_rate.astype(float)
cancer_data['incidence_rate_per_100k'] = cancer_data.incidence_rate_per_100k.replace('*', np.nan)
cancer_data['incidence_rate_per_100k'] = cancer_data.incidence_rate_per_100k.replace('¶', np.nan)
cancer_data['incidence_rate_per_100k'] = cancer_data.incidence_rate_per_100k.str.replace(" #  ", "")
cancer_data['incidence_rate_per_100k'] = cancer_data.incidence_rate_per_100k.astype(float)
cancer_data['recent_trend'] = cancer_data.recent_trend.replace('*', "Nan")
cancer_data['recent_trend'] = cancer_data.recent_trend.replace('¶', "Nan")
cancer_data['avg_annual_count'] = cancer_data.avg_annual_count.str.replace(",", "")
cancer_data['avg_annual_count'] = cancer_data.avg_annual_count.str.replace(" or fewer", "")
cancer_data['avg_annual_count'] = cancer_data.avg_annual_count.replace("¶", 0)
cancer_data['avg_annual_count'] = cancer_data.avg_annual_count.astype(int)


In [ ]:
cancer_data.loc[cancer_data.five_year_incidence_change_rate == "*"]
cancer_data['five_year_incidence_change_rate'] = cancer_data.five_year_incidence_change_rate.replace('*', np.nan)
cancer_data['five_year_incidence_change_rate'] = cancer_data.five_year_incidence_change_rate.replace('¶', np.nan)
cancer_data['five_year_incidence_change_rate'] = cancer_data.five_year_incidence_change_rate.astype(float)
cancer_data['incidence_rate_per_100k'] = cancer_data.incidence_rate_per_100k.replace('*', np.nan)
cancer_data['incidence_rate_per_100k'] = cancer_data.incidence_rate_per_100k.replace('¶', np.nan)
cancer_data['incidence_rate_per_100k'] = cancer_data.incidence_rate_per_100k.str.replace(" #  ", "")
cancer_data['incidence_rate_per_100k'] = cancer_data.incidence_rate_per_100k.astype(float)
cancer_data['recent_trend'] = cancer_data.recent_trend.replace('*', "Nan")
cancer_data['recent_trend'] = cancer_data.recent_trend.replace('¶', "Nan")
cancer_data['avg_annual_count'] = cancer_data.avg_annual_count.str.replace(",", "")
cancer_data['avg_annual_count'] = cancer_data.avg_annual_count.str.replace(" or fewer", "")
cancer_data['avg_annual_count'] = cancer_data.avg_annual_count.replace("¶", 0)
cancer_data['avg_annual_count'] = cancer_data.avg_annual_count.astype(int)


In [ ]:
def state(c):
    if c['stateFIPS'] == 1:
        return "Alabama"
    elif c['stateFIPS'] == 2:
        return "Alaska"
    elif c['stateFIPS'] == 3:
        return "American Samoa"
    elif c['stateFIPS'] == 4:
        return "Arizona"
    elif c['stateFIPS'] == 5:
        return "Arkansas"
    elif c['stateFIPS'] == 6:
        return "California"
    elif c['stateFIPS'] == 8:
        return "Colorado"
    elif c['stateFIPS'] == 9:
        return "Connecticut"
    elif c['stateFIPS'] == 10:
        return "Delaware"
    elif c['stateFIPS'] == 11:
        return "DC"
    elif c['stateFIPS'] == 12:
        return "Florida"
    elif c['stateFIPS'] == 13:
        return "Georgia"
    elif c['stateFIPS'] == 14:
        return "Guam"
    elif c['stateFIPS'] == 15:
        return "Hawaii"
    elif c['stateFIPS'] == 16:
        return "Idaho"
    elif c['stateFIPS'] == 17:
        return "Illinois"
    elif c['stateFIPS'] == 18:
        return "Indiana"
    elif c['stateFIPS'] == 19:
        return "Iowa"
    elif c['stateFIPS'] == 20:
        return "Kansas"
    elif c['stateFIPS'] == 21:
        return "Kentucky"
    elif c['stateFIPS'] == 22:
        return "Louisiana"
    elif c['stateFIPS'] == 23:
        return "Maine"
    elif c['stateFIPS'] == 24:
        return "Maryland"
    elif c['stateFIPS'] == 25:
        return "Massachusetts"
    elif c['stateFIPS'] == 26:
        return "Michigan"
    elif c['stateFIPS'] == 27:
        return "Minnesota"
    elif c['stateFIPS'] == 28:
        return "Mississippi"
    elif c['stateFIPS'] == 29:
        return "Missouri"
    elif c['stateFIPS'] == 30:
        return "Montana"
    elif c['stateFIPS'] == 31:
        return "Nebraska"
    elif c['stateFIPS'] == 32:
        return "Nevada"
    elif c['stateFIPS'] == 33:
        return "New Hampshire"
    elif c['stateFIPS'] == 34:
        return "New Jersey"
    elif c['stateFIPS'] == 35:
        return "New Mexico"
    elif c['stateFIPS'] == 36:
        return "New York"
    elif c['stateFIPS'] == 37:
        return "North Carolina"
    elif c['stateFIPS'] == 38:
        return "North Dakota"
    elif c['stateFIPS'] == 39:
        return "Ohio"
    elif c['stateFIPS'] == 40:
        return "Oklahoma"
    elif c['stateFIPS'] == 41:
        return "Oregon"
    elif c['stateFIPS'] == 42:
        return "Pennsylvania"
    elif c['stateFIPS'] == 43:
        return "Puerto Rico"
    elif c['stateFIPS'] == 44:
        return "Rhode Island"
    elif c['stateFIPS'] == 45:
        return "South Carolina"
    elif c['stateFIPS'] == 46:
        return "South Dakota"
    elif c['stateFIPS'] == 47:
        return "Tennessee"
    elif c['stateFIPS'] == 48:
        return "Texas"
    elif c['stateFIPS'] == 49:
        return "Utah"
    elif c['stateFIPS'] == 50:
        return "Vermont"
    elif c['stateFIPS'] == 51:
        return "Virginia"
    elif c['stateFIPS'] == 53:
        return "Washington"
    elif c['stateFIPS'] == 54:
        return "West Virginia"
    elif c['stateFIPS'] == 55:
        return "Wisconsin"
    elif c['stateFIPS'] == 56:
        return "Wyoming"
    elif c['stateFIPS'] == 72:
        return "Puerto Rico"
    else:
        return "Other"
    
cancer_data['state'] = cancer_data.apply(state, axis = 1)
cancer_data['specific_county'] = cancer_data.county.str.cat(cancer_data.state, sep = " ")
cancer_data['specific_county'] = cancer_data.specific_county.astype(str)
cancer_data = cancer_data.loc[cancer_data.recent_trend != 'Nan']


In [ ]:
merged = pd.merge(cancer_data, air_pollution_data, on = ["specific_county"], how = "inner")
merged = merged.drop('Unnamed: 0', 1)
print("The shape of the merged dataset is: ") 
merged.shape

In [ ]:
sns.heatmap(merged.isnull())

In [ ]:
merged.describe().T.style.background_gradient()

In [ ]:
px.histogram(merged, x = 'recent_trend', color = 'state')

In [ ]:
number_columns = ['incidence_rate_per_100k', 'avg_annual_count', 'five_year_incidence_change_rate', 
            'bad_days', 'average_temperature', 'average_daily_peak_pollution', 
            'maximum_pollution_level', 'worst_day_pollution', 'average_o3', 'average_aqi', 'average_daily_peak_pollution_time']

for column in number_columns:
    fig, axes = plt.subplots(1, 2, figsize = (15, 5))
    sns.histplot(x = merged[column], ax = axes[0]).set(title = column)
    sns.boxplot(x = merged[column], ax = axes[1]).set(title = column)
    fig.show()

In [ ]:
sns.heatmap(merged.corr())

In [ ]:
correlated = merged.corr().abs().unstack().sort_values(ascending = False)

print(correlated[17:37])

In [ ]:
print(correlated[65:100])

In [ ]:
fig = px.histogram(merged, x = 'state', y = ['avg_annual_count', 'incidence_rate_per_100k'], barmode = 'group', title = "Cancer By State")
fig.update_layout(xaxis = {'categoryorder': 'total descending'})
fig.show()

In [ ]:
sns.pairplot(merged, vars = ['incidence_rate_per_100k', 'bad_days', 'avg_annual_count', 
                             'worst_day_pollution', 'average_daily_peak_pollution', 'maximum_pollution_level', 
                             'average_aqi', 'average_o3', 'average_temperature'])

In [ ]:
px.scatter(merged, x = 'average_o3', y = 'incidence_rate_per_100k', facet_col = 'recent_trend', color = 'average_aqi', trendline = 'lowess')

In [ ]:
px.scatter(merged, x = 'average_aqi', y = 'incidence_rate_per_100k', facet_col = 'recent_trend', color = 'average_aqi', trendline = 'lowess')

In [ ]:
px.scatter(merged, x = 'bad_days', y = 'incidence_rate_per_100k', color = 'average_aqi', facet_col = 'recent_trend', trendline = 'lowess')

In [ ]:
px.scatter(merged, x = 'maximum_pollution_level', y = 'incidence_rate_per_100k', facet_col = 'recent_trend', color = 'average_aqi', trendline = 'lowess')

In [ ]:
px.density_heatmap(merged, x = 'incidence_rate_per_100k', y = 'maximum_pollution_level')

In [ ]:
#Gratitude to Dan Becker
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

features = ['avg_annual_count', 'bad_days',  'average_daily_peak_pollution', 
            'maximum_pollution_level', 'worst_day_pollution', 'average_o3', 
            'average_aqi', 'average_daily_peak_pollution_time']
X = merged[features]
y = merged.incidence_rate_per_100k
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

my_model = RandomForestRegressor(n_estimators = 190, max_depth = 8, random_state = 1)
my_model.fit(X_train, y_train)

scores = -1 * cross_val_score(my_model, X, y, cv=7, scoring = 'neg_mean_absolute_error')

print("Average MAE: ", scores.mean())

feature_importances = pd.Series(my_model.feature_importances_, index=X_train.columns).sort_values(ascending=False)

sns.barplot(x = feature_importances, y = feature_importances.index)